# The Use of Arrays on Glaciers
   <img src="pictures/Grenzgletscher_oben.jpg" alt="Grenzgletscher 24" width="600"/>  <img src="pictures/Upper_3700.jpg" alt="Grenzgletscher 3700m" width="600"/>

<b> Developed by J. Wassermann modified by N. Richels </b> 


In this pretty dense notebook, we try to do better in the sense 
that we want also to obtain features which are seperating different types of events
If you know nothing about the signals, their characteristics and also their 
locations its often prefferable to start with arrays of seismic sensors. <br>
A seismic array <b>(@Heiner: yes, also 6C would be possible)</b> allows you to decompose 
the wavefield into its components and get first insights in the velocity structure 
as well as location of the corresponding events. In case of tremor like signals arrays 
are often the only possibility to estimate the source location and possible change of 
the source location.

There are various arraytools on the market - also a standard module in obspy, we use for your (and my) convenience a differrent module which is the result of
a <br> <b>Skience Workshop in 2014(!)</b>. <br> You might visit the tutorial as well (be aware still a lot of code polishing needed!) <br> http://github.com/jwassermann/obspy_arraytools

In [ ]:
import os
import sys

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from obspy import Stream,Trace,UTCDateTime,read_inventory 
from obspy.core.inventory.inventory import Inventory
from obspy.core import AttribDict

import matplotlib.dates as mdates
from matplotlib.dates import julian2num
import matplotlib.cm as cm


from obspy.clients.filesystem import sds
from obspy.signal.invsim import cosine_taper

import obspy_arraytools as AA

SDS_DATA_PATH = "/Users/jowa/Skience2026_Data/data_sds"


In [ ]:
sds_rootdir = SDS_DATA_PATH
arraystats = ["XG.UP1..GLZ","XG.UP2..GLZ","XG.UP3..GLZ","XG.UP4..GLZ","XG.UP5..GLZ","XG.UP6..GLZ"]
output_path = "./Grenzgletscher_fk"
figure_path = "./Grenzgletscher_fk/figure"
fl=10.0 #lower frequency 
fh=80.00 #upper frequency limit
win_len=2.0  #windowlenth of sliding window in seconds
win_frac=0.1 #step width of sliding window
sll_x=-0.5 #lower bound of slowness s/km
slm_x=0.5 #upper bound of slowness s/km
sll_y=-0.5 #ditto but y axis
slm_y=0.5 #ditto
sl_s=0.025 #grid with for search area
thres_rel = 0.6 #semblance value for trigger

client = sds.Client(sds_root=sds_rootdir)
#Let's analyse just two hours of data
ts= t = UTCDateTime("2024-03-20T00:00:00")
e = UTCDateTime("2024-03-20T02:00:00")

## Note
#### we first create outputs from the array analysis on which we later trigger
#### The following cell is creating a new sds directory an appends hourly segments to existing files
#### If you re-run this cell you have to <b> delete </b>the corresponding folder, otherwise you double your data

In [ ]:
#check if you realy want doing this
!rm -rf "%s/%04d"%(out_path,ts.year)

In [ ]:
while (ts+3600) < e:
    start = (ts)
    end = (ts+3600)
    ts += 3600
    try:
        sz = Stream()
        inv= Inventory()
        i = 0
        #reading in data and metadata of the array(s)
        for station in arraystats:
            net,stat,loc,chan=station.split('.')
            tr = client.get_waveforms(network=net,station=stat,location=loc,channel=chan, starttime=start, endtime=end)
            ii = read_inventory("./stationxml/station_%s_%s.xml"%(net,stat))
            tr.merge()
            if tr[0].stats.endtime < end or tr[0].stats.starttime > start:
                print("trace ",tr, "too short")
            else: 
                print(tr)
                sz += tr
                inv += ii
        sz.merge()
        sz.detrend("linear")
        sz.attach_response(inv)
        
        #we restrict ourself to the vertical components.... at least for now
        vc = sz.select(component="Z")
        
        #veeeery convenient function 
        # handles all geometry issues of the array
        array = AA.SeismicArray("",inv)
        array.inventory_cull(vc)
        
        print("Center of gravity: ",array.center_of_gravity)
        
        outray = 0. 
        
        #we use here a covariance based FK-analysis
        outray = array.fk_analysis(vc, frqlow=fl, frqhigh=fh, prefilter=True,\
                         static3d=False, array_response=False,vel_corr=4.8, wlen=win_len,\
                         wfrac=win_frac,sec_km=True,
                         slx=(sll_x,slm_x),sly=(sll_y,slm_y),
                         sls=sl_s)

        trace1 = Trace(data=outray.max_rel_power)
        trace1.stats.channel = 'REL'
        out = outray.max_rel_power
        trace2 = Trace(data=outray.max_abs_power)
        trace2.stats.channel = 'ABS'
        out = np.vstack([out,outray.max_abs_power])

        trace3 = Trace(data=outray.max_pow_baz)
        trace3.stats.channel = 'BACK'
        out = np.vstack([out,outray.max_pow_baz])

        trace4 = Trace(data=outray.max_pow_slow)
        trace4.stats.channel = 'SLOW'
        out = np.vstack([out,outray.max_pow_slow])

        #saving f-k analysis results into mseed file
        fk = Stream()
        tr = Trace()

        delta = outray.timestep

        tr.stats.network = outray.inventory.networks[0].code
        tr.stats.station = outray.inventory.networks[0][0].code
        tr.stats.channel = "ZGC"
        tr.stats.location = ""
        tr.data = outray.max_rel_power
        tr.stats.starttime = outray.starttime
        tr.stats.delta = delta

        fk += tr

        tr = Trace()
        tr.stats.network = outray.inventory.networks[0].code
        tr.stats.station = outray.inventory.networks[0][0].code
        tr.stats.channel = "ZGI"
        tr.stats.location = ""
        tr.stats.starttime = outray.starttime
        tr.data = outray.max_abs_power
        tr.stats.delta = delta

        fk += tr
        tr = Trace()
        tr.stats.network = outray.inventory.networks[0].code
        tr.stats.station = outray.inventory.networks[0][0].code
        tr.stats.channel = "ZGS"
        tr.stats.location = ""
        tr.stats.starttime = outray.starttime
        tr.data = outray.max_pow_baz
        tr.stats.delta = delta

        fk += tr

        tr = Trace()
        tr.stats.network = outray.inventory.networks[0].code
        tr.stats.station = outray.inventory.networks[0][0].code
        tr.stats.channel = "ZGA"
        tr.stats.location = ""
        tr.stats.starttime = outray.starttime
        tr.data = outray.max_pow_slow
        tr.stats.delta = delta

        fk += tr

        myday = "%03d"%fk[0].stats.starttime.julday

        pathyear = str(fk[0].stats.starttime.year)
        # open catalog file in read and write mode in case we are continuing d/l,
        # so we can append to the file
        mydatapath = os.path.join(output_path, pathyear)
        # create datapath 
        if not os.path.exists(mydatapath):
            os.mkdir(mydatapath)

        mydatapath = os.path.join(mydatapath, fk[0].stats.network)
        if not os.path.exists(mydatapath):
            os.mkdir(mydatapath)

        mydatapath = os.path.join(mydatapath, fk[0].stats.station)

        # create datapath 
        if not os.path.exists(mydatapath):
                os.mkdir(mydatapath)


        for tr in fk:
            print("saving to " + mydatapath)
            print(tr)
            mydatapathchannel = os.path.join(mydatapath,tr.stats.channel + ".D")

            if not os.path.exists(mydatapathchannel):
                os.mkdir(mydatapathchannel)

            netFile = tr.stats.network + "." + tr.stats.station +  "." + tr.stats.location + "." + tr.stats.channel+ ".D." + pathyear + "." + myday
            netFileout = os.path.join(mydatapathchannel, netFile)

            # try to open File
            print(netFileout)
            try:
                netFileout = open(netFileout, 'ab')
            except:
                netFileout = open(netFileout, 'w')
            tr.write(netFileout , format='MSEED',encoding="FLOAT64")
            netFileout.close()

        #print(outray)
        # Plot FK
        labels = ['ref','rel.power', 'abs.power', 'baz', 'slow']
        xlocator = mdates.AutoDateLocator()
        fig = plt.figure()
        alphas = out[0,:]
        condition1 = (out[0,:] < thres_rel)
        condition2 = (out[3,:] > 0.4) 
        tt = np.ma.masked_array(fk[0].times("matplotlib"),mask=condition1)
        tt = np.ma.masked_array(tt,mask=condition2)
        axis = []

        for i, lab in enumerate(labels):
            try:
                if i == 0:
                    ax = fig.add_subplot(5, 1, i + 1,sharex=None)
                    ax.plot(vc[0].times("matplotlib"),vc[0].data)
                else:
                    ax = fig.add_subplot(5, 1, i + 1,sharex=axis[0])
                    mask_v = np.ma.masked_array(out[i-1,:],mask=condition1)
                    mask_v = np.ma.masked_array(mask_v,mask=condition2)
                    ax.scatter(tt,mask_v, c=out[0,:], alpha=alphas,
                       edgecolors='none', cmap=cm.viridis_r)
                    ax.set_ylabel(lab)
                    ax.set_ylim(mask_v.min()-0.1, mask_v.max()+0.1)
                    ax.xaxis.set_major_locator(xlocator)
                    ax.xaxis.set_major_formatter(mdates.AutoDateFormatter(xlocator))
                axis.append(ax)
            except Exception as er:
                sys.stderr.write("Error:" + str(er))
                traceback.print_exc()
        fig.suptitle( 'jane-fk %s' % ( start ))
        fig.autofmt_xdate()
        fig.subplots_adjust(left=0.15, top=0.95, right=0.95, bottom=0.2, hspace=0)
        plt.savefig("%s/FK-%s.png"%(figure_path,start.strftime('%Y-%m-%dT%H')))
        #plt.show()
        plt.close("all")
    except:
        print(start)
        continue

        

# Now we trigger ....

In [ ]:
from obspy.core import UTCDateTime, Stream, AttribDict
from obspy import read_inventory
from obspy.clients.filesystem import sds
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
import numpy as np
import logging
import os
from datetime import timedelta
import matplotlib as mpl
from matplotlib.gridspec import GridSpec

par = AttribDict()
par.filter = AttribDict(type="bandpass", freqmin=5.0, freqmax=30.0,
                            corners=2, zerophase=True)

# Set better default style for matplotlib
plt.style.use('seaborn-v0_8')
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['font.family'] = 'sans-serif'

# Add logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

out_file = "./Grenzgletscher_fk/fk_trigger.csv"
sds_rootdir = "./Grenzgletscher_fk/"
save_path = "./Grenzgletscher_fk/trig_figs/"
data_dir = SDS_DATA_PATH

# Create directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

cl = sds.Client(sds_root=sds_rootdir)
clw = sds.Client(sds_root=data_dir)

ostart = start = UTCDateTime(2024, 3, 20, 00)
end = UTCDateTime(2024, 3, 21)
trig_times = []

# Debug info
logger.info(f"Starting trigger detection from {ostart} to {end}")
trigger_count = 0

<br>
For triggering we simply use the semblance values (i.e. multi-trace coherence) and slowness (i.e. body waves) <br>
as measure of an indication of an incoming event
<br>

In [ ]:
#Thresholds for detection
relp_threshold = 0.6
slow_threshold = 0.5
min_trigger_separation = 2  # Minimum seconds between triggers

last_trigger_abs_time = None

In [ ]:
while start + 1*3600 < end:
    endy = start + 1*3600
    try:
        logger.info(f"Processing window: {start} - {endy}")
        st = cl.get_waveforms(network="XG", station="UP1", location="", channel="ZG?", starttime=start, endtime=endy)
        st.merge()
        
        # Check for required channels
        channels = [tr.stats.channel for tr in st]
        if "ZGC" not in channels or "ZGA" not in channels:
            logger.warning(f"Missing required channels. Available: {channels}")
            start += 1*3600
            continue
        
        relp = st.select(channel="ZGC")[0]
        slow = st.select(channel="ZGA")[0]
        
        window_triggers = 0
        
        for i in range(1, relp.stats.npts):
            # Check if conditions are met
            if relp.data[i] > relp_threshold and slow.data[i] < slow_threshold:
                # Check for state transition
                if i > 5 and (
                    (np.mean(relp.data[i-1:i]) < relp_threshold) or 
                    (np.mean(slow.data[i-1:i]) > slow_threshold)
                ):
                    trig_time = relp.times(reftime=ostart)[i]
                    abs_time = ostart + trig_time
                    
                    # Check minimum separation
                    if last_trigger_abs_time is None or (abs_time - last_trigger_abs_time) > min_trigger_separation:
                        trig_times.append(trig_time)
                        logger.info(f"Trigger detected at index {i}: relative time={trig_time}, absolute time={abs_time}")
                        window_triggers += 1
                        last_trigger_abs_time = abs_time
                    else:
                        logger.info(f"Skipping close trigger at {abs_time} (too close to previous)")
        logger.info(f"Found {window_triggers} triggers in this window")
        trigger_count += window_triggers
        start += 1*3600
    except Exception as e:
        logger.error(f"Error processing window {start}-{endy}: {e}")
        start += 1*3600
        continue

logger.info(f"Total triggers detected: {trigger_count}")
logger.info(f"Writing triggers to {out_file}")

In [ ]:
# Add validation before writing to CSV
valid_triggers = []
for j in trig_times:
    trigger_time = ostart + j
    valid_triggers.append(trigger_time)

# Write triggers to file
with open(out_file, "w") as fo:
    for trigger_time in valid_triggers:
        fo.write("%s\n" % trigger_time)

logger.info(f"Written {len(valid_triggers)} triggers to CSV file")

#### Now extensive plotting is done ... We will see later why

In [ ]:
# Process individual events
all_baz = []
all_slow = []
all_rp = []

# Define the global colormap
global_cmap = plt.cm.viridis

# Define wave velocities for incidence angle calculation
p_velocity = 3.8  # km/s for P-waves
s_velocity = 1.8  # km/s for S-waves

logger.info(f"Processing {len(trig_times)} individual events")

#Track successful plot creation
successful_plots = 0
failed_plots = 0
for j_idx, j in enumerate(trig_times):
    try:
        event_time = ostart + j
        logger.info(f"Processing event {j_idx+1}/{len(trig_times)} at {event_time}")
        
        # Get waveform data with a window around the event time
        try:
            st = clw.get_waveforms(network="XG", station="UP1", location="", channel="??Z", 
                                   starttime=(event_time-1), endtime=event_time+10)
            ar = cl.get_waveforms(network="XG", station="UP1", location="", channel="ZG?", 
                                  starttime=(event_time-1), endtime=event_time+10)
        except Exception as data_error:
            logger.error(f"Failed to get waveform data for event at {event_time}: {data_error}")
            failed_plots += 1
            continue
        
        # Check if valid data
        if len(st) == 0 or len(ar) == 0:
            logger.warning(f"No data found for event at {event_time}, skipping")
            failed_plots += 1
            continue
            
        # Check for required channels
        ar_channels = [tr.stats.channel for tr in ar]
        if "ZGC" not in ar_channels or "ZGA" not in ar_channels or "ZGS" not in ar_channels:
            logger.warning(f"Missing required array channels for event at {event_time}, skipping. Available: {ar_channels}")
            failed_plots += 1
            continue
            
        # Check if there's a valid vertical component
        if not st.select(component="Z"):
            logger.warning(f"No vertical component found for event at {event_time}, skipping")
            failed_plots += 1
            continue
            
        # Process waveforms
        st.detrend("linear")
        st.taper(type='cosine', max_percentage=0.05)
        st.filter(**par.filter)
        # Extract data
        if ar.select(channel="ZGC"):
            rel_power = ar.select(channel="ZGC")[0].data
            all_rp.append(rel_power)
        else:
            logger.warning(f"Missing ZGC channel for event at {event_time}")
            failed_plots += 1
            continue
            
        if ar.select(channel="ZGS"):
            baz = ar.select(channel="ZGS")[0].data
            all_baz.append(baz)
        else:
            logger.warning(f"Missing ZGS channel for event at {event_time}")
            failed_plots += 1
            continue
            
        if ar.select(channel="ZGA"):
            slow = ar.select(channel="ZGA")[0].data
            all_slow.append(slow)
        else:
            logger.warning(f"Missing ZGA channel for event at {event_time}")
            failed_plots += 1
            continue
            
        # Calculate incidence angles based on slowness values
        incidence_angles = []
        wave_types = []
        for s in slow:
            if s < 0.3:  # P-wave region
                sin_i = min(p_velocity * s, 0.99)
                angle = np.degrees(np.arcsin(sin_i))
                wave_type = "P"
            elif s <= 0.6:  # S-wave region
                if s_velocity * s > 0.99:
                    # Scale between 60-85 degrees based on the slowness value
                    angle = 60 + 25 * (s - 0.2) / 0.3
                else:
                    sin_i = min(s_velocity * s, 0.99)
                    angle = np.degrees(np.arcsin(sin_i))
                wave_type = "S"
            else:
                # For values outside velocity model assumptions
                angle = np.nan
                wave_type = "Unknown"
                
            incidence_angles.append(angle)
            wave_types.append(wave_type)
        
        # Convert to numpy arrays
        incidence_angles = np.array(incidence_angles)
        wave_types = np.array(wave_types)
        
        # Close any existing figures
        plt.close('all')
        
        # Create figure
        fig = plt.figure(figsize=(12, 16), dpi=100)
        fig.suptitle(f"Seismic Event Analysis - {event_time.strftime('%Y-%m-%d %H:%M:%S')}", 
                    fontsize=16, fontweight='bold', y=0.98)
        
        # Create a grid layout with 3 rows and 2 columns
        gs = plt.GridSpec(3, 2, figure=fig, height_ratios=[1, 1, 1], width_ratios=[1, 1],
                          hspace=0.35, wspace=0.35)
        common_time_limits = None
        if st.select(component="Z"):
            vert_tr = st.select(component="Z")[0]
            time_data = vert_tr.times("matplotlib")
            
            # Get time limits for alignment
            end_time = max(time_data)
            start_time = min(time_data)
            
            # Common time limits for all time-based plots
            common_time_limits = [start_time, end_time]
            
            # Create the time locator for all plots
            seconds_locator = mdates.SecondLocator(interval=1)
            seconds_formatter = mdates.DateFormatter('%H:%M:%S')
        else:
            logger.warning("No vertical component data for establishing time limits")
            failed_plots += 1
            continue
        
        # PLOT 1: Seismogram - Row 1, Col 1 (Upper Left)
        axtrace = fig.add_subplot(gs[0, 0])
        # Plot the vertical component data
        if st.select(component="Z"):
            axtrace.plot(time_data, vert_tr.data, 'k', linewidth=1.2)
            axtrace.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
            axtrace.set_ylabel('Amplitude [mm/s]', color='k', fontsize=11, fontweight='bold')
            axtrace.set_title('Vertical Component Waveform', fontsize=12, fontweight='bold', pad=10)
            
            # Set x-axis ticks every second
            axtrace.xaxis.set_major_locator(seconds_locator)
            axtrace.xaxis.set_major_formatter(seconds_formatter)
            
            # Set time limits
            axtrace.set_xlim(common_time_limits)
            
            # Enhance grid for seismogram
            axtrace.grid(True, which='both', axis='x', color='gray', alpha=0.5, linestyle='-')
            axtrace.grid(True, which='major', axis='y', color='gray', alpha=0.5, linestyle='-')
            
            # Rotate time labels
            plt.setp(axtrace.xaxis.get_majorticklabels(), rotation=45, ha='right')
        else:
            logger.warning("No vertical component data for trace plot")
            failed_plots += 1
            continue
            
        # PLOT 2: Polar Plot - Row 1, Col 2 (Upper Right)
        polar_ax = fig.add_subplot(gs[0, 1], projection='polar')
        # Check for the necessary data
        if len(baz) > 0 and len(slow) > 0 and len(rel_power) > 0:
            # Convert backazimuth to radians
            baz_rad = np.radians(baz)
            baz_rad[baz_rad < 0] += 2*np.pi
            baz_rad[baz_rad > 2*np.pi] -= 2*np.pi
            
            # Create 2D histogram for polar plot
            N = int(360./5.)  # 5-degree bins
            abins = np.arange(N + 1) * 2*np.pi / N
            sbins = np.linspace(0, 0.4, 20) 
            
            hist, baz_edges, sl_edges = np.histogram2d(baz_rad, slow, bins=[abins, sbins], weights=rel_power)
            
            # Create meshgrid for pcolormesh
            A, S = np.meshgrid(abins, sbins)

            polar_ax.set_theta_zero_location("N")
            polar_ax.set_theta_direction(-1)
            
            # Use pcolormesh for polar plot with the global colormap
            pcm = polar_ax.pcolormesh(A, S, hist.T, cmap=global_cmap, alpha=0.7, shading='auto')
            
            # Improve polar plot settings
            polar_ax.grid(True, linewidth=1.5)
            
            # Add radial labels
            polar_ax.set_rticks([0.1, 0.2, 0.3, 0.4])
            polar_ax.set_rlabel_position(135)
            polar_ax.set_rmax(0.4)
            polar_ax.set_title('Polar Plot: Backazimuth vs. Slowness', fontsize=12, fontweight='bold', pad=15)
        else:
            logger.warning("Missing data for polar plot")
            
        # PLOT 3: Spectrogram - Row 2, Col 1 (Middle Left)
        axspec = fig.add_subplot(gs[1, 0])
        # Get the vertical component data for spectrogram
        if st.select(component="Z"):
            tr = st.select(component="Z")[0]
    
            try:
                # Calculate spectrogram
                specgram = tr.spectrogram(wlen=0.5, per_lap=0.9, show=False, axes=axspec)
        
                # Limit frequency range
                axspec.set_ylim(1, 25)  # Limit frequency to 1-25 Hz
        
                # Set labels and grid
                axspec.set_ylabel('Frequency [Hz]', fontsize=11, fontweight='bold')
                
                # Clear the current x-axis labels and ticks
                axspec.set_xticklabels([])
                axspec.set_xticks([])
                
                # Create secondary axis that matches seismogram time
                ax2 = axspec.twiny()
                ax2.set_xlim(common_time_limits)
                ax2.xaxis.set_major_locator(seconds_locator)
                ax2.xaxis.set_major_formatter(seconds_formatter)
                ax2.xaxis.tick_bottom()
                ax2.xaxis.set_label_position('bottom')
                ax2.tick_params(axis='x', pad=10)
                
                # Rotate time labels
                plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')
                
                # Add horizontal grid lines
                axspec.grid(True, which='major', axis='y', color='gray', alpha=0.01, linestyle='-')
                
                # Add vertical grid lines
                for pos in ax2.get_xticks():
                    axspec.axvline(pos, color='gray', alpha=0.01, linestyle='-')
                
                # Set title above the plot
                axspec.set_title('Spectrogram', fontsize=12, fontweight='bold', pad=10)
            except Exception as e:
                logger.error(f"Error creating spectrogram: {e}")
        else:
            logger.warning("No vertical component data for spectrogram")
            
        # PLOT 4: Incidence Angle - Row 2, Col 2 (Middle Right)
        axangle = fig.add_subplot(gs[1, 1])
        
        # Only plot valid incidence angles
        valid_mask = ~np.isnan(incidence_angles)
        if any(valid_mask):
            # Get matplotlib times for the incidence angle data
            angle_times = ar.select(channel="ZGA")[0].times("matplotlib")
            
            # Calculate point sizes based on relative power if not already defined
            rel_power_norm = rel_power / np.max(rel_power) if np.max(rel_power) > 0 else np.zeros_like(rel_power)
            sizes = 20 + 100 * rel_power_norm
            
            # Use the same marker sizing and coloring scheme as the other plots
            scatter_angle = axangle.scatter(
                angle_times[valid_mask], 
                incidence_angles[valid_mask],
                c=rel_power[valid_mask], 
                cmap=global_cmap, 
                s=sizes[valid_mask], 
                alpha=0.7
            )

            # Add markers to indicate wave type
            p_mask = np.logical_and(valid_mask, np.array(wave_types) == "P")
            if any(p_mask):
                axangle.scatter(
                    angle_times[p_mask], 
                    incidence_angles[p_mask],
                    s=30, alpha=0.7, facecolors='none', edgecolors='blue',
                    linewidth=1.5, marker='o', label='P-wave'
                )
                
            s_mask = np.logical_and(valid_mask, np.array(wave_types) == "S")
            if any(s_mask):
                axangle.scatter(
                    angle_times[s_mask], 
                    incidence_angles[s_mask],
                    s=30, alpha=0.7, facecolors='none', edgecolors='red',
                    linewidth=1.5, marker='s', label='S-wave'
                )
            
            axangle.set_ylabel('Incidence Angle [deg]', fontsize=10, fontweight='bold')
            axangle.set_title('Incidence Angle vs. Time', fontsize=11, fontweight='bold', pad=10)
            
            # Set x-axis ticks every second
            axangle.xaxis.set_major_locator(seconds_locator)
            axangle.xaxis.set_major_formatter(seconds_formatter)
            
            # Set time limits to match seismogram
            axangle.set_xlim(common_time_limits)
            
            # Set reasonable y-limits for the plot
            axangle.set_ylim(0, 90)
            
            # Enhanced grid with lines every second
            axangle.grid(True, which='major', axis='both', color='gray', alpha=0.5, linestyle='-')
            axangle.legend(loc='upper right', fontsize=9)

            # Rotate time labels
            plt.setp(axangle.xaxis.get_majorticklabels(), rotation=45, ha='right')
        else:
            logger.warning("No valid incidence angle data for plot")
        
        # PLOT 5: Backazimuth - Row 3, Col 1 (Lower Left)
        axbaz = fig.add_subplot(gs[2, 0])
        
        # Get matplotlib times for the backazimuth data
        if ar.select(channel="ZGS") and len(ar.select(channel="ZGS")[0].data) > 0:
            baz_times = ar.select(channel="ZGS")[0].times("matplotlib")
            
            # Check for matching data lengths
            if len(baz_times) == len(baz) and len(baz) == len(rel_power):
                scatter_baz = axbaz.scatter(baz_times, baz, 
                           c=rel_power, cmap=global_cmap, s=sizes, alpha=0.7)
                axbaz.set_ylabel('Backazimuth [deg]', fontsize=11, fontweight='bold')
                axbaz.set_xlabel('Time (UTC)', fontsize=11, fontweight='bold')
                axbaz.set_ylim(0, 360)
                axbaz.set_yticks([0, 90, 180, 270, 360])
                axbaz.set_title('Backazimuth vs. Time', fontsize=12, fontweight='bold', pad=10)
                
                # Set x-axis ticks every second
                axbaz.xaxis.set_major_locator(seconds_locator)
                axbaz.xaxis.set_major_formatter(seconds_formatter)
                
                # Set time limits to match seismogram
                axbaz.set_xlim(common_time_limits)
                
                # Enhanced grid with lines
                axbaz.grid(True, which='major', axis='both', color='gray', alpha=0.5, linestyle='-')
                
                # Rotate time labels
                plt.setp(axbaz.xaxis.get_majorticklabels(), rotation=45, ha='right')
            else:
                logger.warning(f"Data length mismatch in backazimuth plot")
        else:
            logger.warning("No backazimuth data available for plot")
        # PLOT 6: Slowness - Row 3, Col 2 (Lower Right)
        axslow = fig.add_subplot(gs[2, 1])
        
        # Add horizontal line for the threshold
        axslow.axhline(y=slow_threshold, color='r', linestyle='--', alpha=0.7, 
                       label=f'Threshold ({slow_threshold})')
        
        # Get matplotlib times for the slowness data
        if ar.select(channel="ZGA") and len(ar.select(channel="ZGA")[0].data) > 0:
            slow_times = ar.select(channel="ZGA")[0].times("matplotlib")
            
            # Check for matching data lengths
            if len(slow_times) == len(slow) and len(slow) == len(rel_power):
                # Use scatter for slowness, sized by rel_power like backazimuth
                scatter_slow = axslow.scatter(slow_times, slow, 
                            c=rel_power, cmap=global_cmap, s=sizes, alpha=0.7)
                
                axslow.set_ylabel('Slowness [s/km]', fontsize=11, fontweight='bold')
                axslow.set_xlabel('Time (UTC)', fontsize=11, fontweight='bold')
                axslow.set_title('Slowness vs. Time', fontsize=12, fontweight='bold', pad=10)
                
                # Set x-axis ticks every second
                axslow.xaxis.set_major_locator(seconds_locator)
                axslow.xaxis.set_major_formatter(seconds_formatter)
                
                # Set time limits to match seismogram
                axslow.set_xlim(common_time_limits)
                
                # Set y-axis limits
                axslow.set_ylim(0, max(1.0, np.max(slow)*1.1))
                
                # Enhanced grid with lines
                axslow.grid(True, which='major', axis='both', color='gray', alpha=0.5, linestyle='-')
                
                axslow.legend(loc='upper right', fontsize=9)
                
                # Rotate time labels
                plt.setp(axslow.xaxis.get_majorticklabels(), rotation=45, ha='right')
            else:
                logger.warning(f"Data length mismatch in slowness plot")
        else:
            logger.warning("No slowness data available for plot")
        
        # Colorbar for all plots using the same colormap
        if 'pcm' in locals():
            cax = fig.add_axes([0.93, 0.3, 0.02, 0.4])  # Position for vertical colorbar
            cbar = fig.colorbar(pcm, cax=cax)
            cbar.set_label('Relative Power', fontsize=10, fontweight='bold')
        
        # Save figure
        fmt = "png"
        filename = f'{save_path}UP1_{event_time.strftime("%Y%m%d_%H%M%S")}_array.{fmt}'
        

        try:
            plt.savefig(filename, format=fmt, dpi=300)
            logger.info(f"Saved figure: {filename}")
            successful_plots += 1
        except Exception as save_error:
            logger.error(f"Failed to save figure {filename}: {save_error}")
            failed_plots += 1
        
        plt.close("all")
        
    except Exception as e:
        logger.error(f"Error processing event at {event_time}: {e}")
        failed_plots += 1
        plt.close("all")  # Make sure to close all figures even in case of error

logger.info(f"Processing complete! Successfully created {successful_plots} plots, {failed_plots} failed")
logger.info(f"Summary: {len(trig_times)} triggers detected, {len(valid_triggers)} written to CSV, {successful_plots} plots created")